# Multi-Agent RAG Pipline
RAG system contsin (3) major layers:
1. Knowledge Layer (Vector Store)
    - Where documents live, chunked and embdeed (for semantic searching)

2. Retrieval Layer
    - Takes a user query
    - Finds relevant chunks
    - Returns them as context

3. Agent Layer
    Team of LLM-powered agents that:
    - Inerpret question
    - Retrieve context
    - Reason collaboratively
    - Produce anser

## Setup: Import and environemnt
To build a pipeline, we need:
- Vector Store (Chroma DB)
- Embedding Model (OpenAI)
- Chunking logic (break long docs into retrivalbe pieces)
- AutoGen Agents (to reason over retrieved context)

In [1]:
import os
from dotenv import load_dotenv

import chromadb # vectore store backend
from chromadb.config import Settings 

from openai import OpenAI   # for embeddings

from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

import asyncio  # Autogen 0.4 uses aysnc execution
import textwrap # clean chunking

### Load environment and initialize OpenAI + Chroma

In [2]:
# Load env variables
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables.")

In [3]:
# Init openai client (used to generated embeddings)
oa_client = OpenAI(api_key=api_key)

# Init ChromaDB (Persistent, Telemtry Off)
# NOTE: Want local vector store that doesn't "phone home"
client_settings = Settings(anonymized_telemetry=False)
chroma_client = chromadb.PersistentClient(
    path="./chroma_db_multi_agent",
    settings=client_settings
)

### Define OpenAI embedding function for Chroma

In [4]:
EMBEDDING_MODEL = 'text-embedding-3-small'

# NOTE: Wrap embedings in a class as chroma expected an object with __call__ method
class OpenAIEmbeddingFunction:
    def __init__(self, client, model: str):
        self.client = client
        self.model = model

    def __call__(self, input: list[str]) -> list[list[float]]:
        response = self.client.embeddings.create(
            model=self.model,
            input=input,
        )
        return [item.embedding for item in response.data]
    
    # Used for embedding quereis when calling colleciton.query()
    def embed_query(self, input: list[str]) -> list[list[float]]:
        response = self.client.embeddings.create(
            model=self.model,
            input=input
        )
        return [item.embedding for item in response.data]
    
    def name(self) -> str:
        # Chroma requires this for conflict detection
        return f"openai={self.model}"
    
embedding_fn = OpenAIEmbeddingFunction(oa_client, EMBEDDING_MODEL)

### Create Chroma Collection

In [5]:
# chroma_client.delete_collection("rag_collection_multi_agent")     # clean-up (if needed)
collection = chroma_client.get_or_create_collection(
    name="rag_collection_multi_agent",
    embedding_function=embedding_fn,
    metadata={"hnsw:space": "cosine"}   # Use cosine as standard metric for semantic embeddings
)

### Chunking Helper and Indexing Sample Content
(LLM retreive better when documents broken into small, overlapping chunks)

In [6]:
def chunk_text(text: str, chunk_size: int = 300, overlap: int = 50):
    text = textwrap.dedent(text).strip()
    words = text.split()
    chunks = []
    start = 0

    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start = end - overlap # slide with overlap

    return chunks

In [7]:
# Index sample content
kb_text = """ AutoGen is a framework for building multi-agent systems that can collaborate to solve complex tasks. 
It supports tools, function calling, and orchestration patterns for LLM-based agents. 

Retrieval-Augmented Generation (RAG) is a technique where a model retrieves relevant context from an external 
knowledge base (like a vector database) and uses it to ground its responses. 

By combining AutoGen with a vector store like ChromaDB, you can build multi-agent systems that retrieve, 
reason, and respond with up-to-date, domain-specific knowledge. 
"""

# Chunk text
chunks = chunk_text(kb_text, chunk_size=40, overlap=10)

# Generate unique id for each chunk (chroma requries every document to have a unique ID)
ids = [f"chunk-{i}" for i in range(len(chunks))]

# Insert chunked text into Chroma collection 
collection.add(
    documents=chunks,
    ids=ids,
)

# ouput total number of chunks, preview first two chunks
len(chunks), chunks[:2]

(3,
 ['AutoGen is a framework for building multi-agent systems that can collaborate to solve complex tasks. It supports tools, function calling, and orchestration patterns for LLM-based agents. Retrieval-Augmented Generation (RAG) is a technique where a model retrieves relevant context from an',
  'a technique where a model retrieves relevant context from an external knowledge base (like a vector database) and uses it to ground its responses. By combining AutoGen with a vector store like ChromaDB, you can build multi-agent systems that retrieve,'])

(3,
 ['AutoGen is a framework for building multi-agent systems that can collaborate to solve complex tasks. It supports tools, function calling, and orchestration patterns for LLM-based agents. Retrieval-Augmented Generation (RAG) is a technique where a model retrieves relevant context from an',
  'a technique where a model retrieves relevant context from an external knowledge base (like a vector database) and uses it to ground its responses. By combining AutoGen with a vector store like ChromaDB, you can build multi-agent systems that retrieve,'])

## Build Multi-Agent Team
This multi-agent team will have the following agents: researcher, writer, critic

### Create Model Client

In [8]:
# NOTE: Did previously, restating for clarity
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

### Create the Agents
Key things to note about agents
- Like movie star actors, agents need clear-defined roles

In [ ]:


writer = AssistantAgent(
    name="writer",
    model_client=model_client,
    system_message=(
        "You are a writing-focused agent. "
        "Your job is to synthesize a clear, accurate, grounded answer "
        "using the retrieved context and the researcher's plan. "
        "Write with clarity and precision"
    )
)

critic = AssistantAgent(
    name="critic",
    model_client=model_client,
    system_message=(
        "You are a critical evaluator. "
        "Your job is to review the writer's answer for correctness, "
        "grounding in retrieved context, and clarity. "
        "Suggest improvements when necessary"
    )
)

### Define Agent-to-Agent Messaging

In [10]:

# NOTE: Change in how communicate with agents
'''
Chat helper function

Instead of having to run agents like:
`await researcher.run(task="Here is my plan", recipient=writer)`

Can NOW just say:
`await chat(researcher, writer, "Here is my plan")`
'''
async def chat(sender, receiver, message: str):
    # Ask the sender agent to run a task
    result = await sender.run(
        task=message,   # what sender is saying
        receiver=receiver   # who the sender is talkign to
    )

    # Return the result of that interaction
    return result

In [11]:
# Sanity Check 
result = await researcher.run(task="Explain your role in one sentence.")
print(result.messages[-1].content)

My role is to analyze users' queries, gather relevant context, and propose a comprehensive plan for answering their questions with precision and clarity.


As a researcher-focused agent, my role is to analyze the user's query, retrieve relevant context and information, and propose a comprehensive plan for addressing their question or research needs.

In [12]:
async def chat_stream(sender, receiver, message: str):
    """
    Stream messages from one agent to another.
    Yields each from as it arrives, and returns the final result.
    """
    final_result = None

    async for frame in sender.run_stream(
        task=message,
        recipient=receiver
        ):
        # Each frame is a dict with role + content
        role = frame.get("role", "assistant")
        content = frame.get("content", "")

        # Display the streamed content
        print(f"[{sender.name} -> {receiver.name}] {role}: {content}")

        # Final frame contains full result object
        if frame.get("event") == "on_complete":
            final_result = frame["result"]

        return final_result


## Buld Multi-AGent RAG Pipeline (Retrieval + Collaboration + Revision)
In the previous single RAG system, it simply retreived the context and answered questions
In the multi-agent RAG pipeline, it does the following:
1. Retrival - Find relevant chunks from vector store
2. Researcher - Interprets question, analyzes retrieved context, proposes a plan
3. Writer - Synthesizes grounded answer using plan + retrieved context
4. Critic - Evaluates answer for correctness, grounding, clarity, hallucinations
5. Writer (revision) - Improves answer based on critic's feeback

### Build a Retrieval Helper for the Pipeline
*Similar to notebook #1, but return BOTH chunks and metadata so researcher can reason about them*


In [13]:
# Retrival Helper Function
def retrieve_context(query: str, k: int = 4):
    """
    Retrieve the top-k most relevant chunks from ChromaDB. 
    Returns both the chunks and their IDs for traceability.
    """
    results = collection.query(
        query_texts=[query],
        n_results=k
    )

    chunks = results["documents"][0]
    ids = results["ids"][0]

    print(chunks)
    print(ids)

    return chunks, ids

### Multi-Agent RAG Pipeline Orchestrator
Function coordinates entire multi-agent workflow

In [ ]:
async def multi_agent_rag(query: str):
    """
    Full multi-agent RAG workflow:
    1. Retrieve context
    2. Researcher analyzes question + context
    3. Writer drafts answer
    4. Critic reviews answer
    5. Writer revises
    """

    # NOTE: For the prompts, they are formated 

    # Step 1: Retrieve context
    chunk, ids = retrieve_context(query)
    context_block = "\n\n".join(
        f"[{ids[i]}] {chunks[i]}" for i in range(len(chunks))
    )

    # Step 2: Researcher analyzes
    researcher_prompt = f"""
    User query:
    {query}

    Retrieved context:
    {context_block}

    Your task:
    - Analyze the question
    - Identify which chunks are relevant
    - Propose a plan for how the writer should answer
    """

    researcher_result = await researcher.run(task=researcher_prompt)
    researcher_plan = researcher_result.messages[-1].content

    # Step 3: Writer drafts answer
    writer_prompt = f"""
    User query:
    {query}

    Retrieved contex:
    {context_block}

    Researchers' plan:
    {researcher_plan}

    Your task:
    - Write a clear, grounded answer
    - Use the retrieved context explicitly
    """

    writer_result = await writer.run(task=writer_prompt)
    draft_answer = writer_result.messages[-1].content

    # Step 4: Critic reviews
    critic_prompt = f"""
    Draft asnwer:
    {draft_answer}

    Retrieved context:
    {context_block}

    Your task:
    - Evaluate correctness
    - Check grounding in context
    - Suggest improvements
    """

    critic_result = await critic.run(task=critic_prompt)
    critique = critic_result.messages[-1].content

    # Step 5: Writer revises
    revision_prompt = f"""
    Original draft:
    {draft_answer}

    Critic's feedback:
    {critique}

    Your task:
    - Produce a revised, improved final answer
    """

    final_result = await writer.run(task=revision_prompt)
    final_answer = final_result.messages[-1].content

    return {
        "context": context_block,
        "plan": researcher_plan,
        "draft": draft_answer,
        "critique": critique,
        "final": final_answer,
        "final_result_obj": final_result
    }

### Stream Final Answer (run_stream)
This version of AutoGen only supports streaming without `recipients=`, 
we stream the final answer to the user, NOT between agents,

In [23]:
# Stream final answer to user 
async def stream_final_answer(query: str):
    result = await multi_agent_rag(query)

    print("\n\n=== FINAL ANSER (streamed) ==\n")

    async for frame in writer.run_stream(task=result["final"]):
        # Some frames may not have content (i.e. start/end events)1
        if hasattr(frame, 'content') and frame.content:
            print(frame.content, end="", flush=True)

    return result

## Test Pipeline

In [24]:
result = await stream_final_answer("What is AutoGen and how does it relate to RAG?")

['AutoGen is a framework for building multi-agent systems that can collaborate to solve complex tasks. It supports tools, function calling, and orchestration patterns for LLM-based agents. Retrieval-Augmented Generation (RAG) is a technique where a model retrieves relevant context from an', 'a technique where a model retrieves relevant context from an external knowledge base (like a vector database) and uses it to ground its responses. By combining AutoGen with a vector store like ChromaDB, you can build multi-agent systems that retrieve,', 'store like ChromaDB, you can build multi-agent systems that retrieve, reason, and respond with up-to-date, domain-specific knowledge.']
['chunk-0', 'chunk-1', 'chunk-2']


=== FINAL ANSER (streamed) ==

AutoGen is a framework specifically designed for building multi-agent systems that can effectively collaborate to solve complex tasks. It supports the integration of tools, function calling, and orchestration patterns, which are crucial for agents u

In [25]:
# NOTE: result in format of user-defined dict (declared above)
print(result["final_result_obj"].messages[-1].content)

AutoGen is a framework specifically designed for building multi-agent systems that can effectively collaborate to solve complex tasks. It supports the integration of tools, function calling, and orchestration patterns, which are crucial for agents utilizing large language models (LLMs). These features enhance the capability of the agents to work together efficiently, addressing challenges that might be insurmountable for a single agent.

Retrieval-Augmented Generation (RAG) is a technique that enhances language model outputs by retrieving relevant context from external knowledge bases, such as vector databases. By incorporating this contextual information, RAG ensures that the models’ responses are grounded in accurate and up-to-date facts, thereby improving both the relevance and precision of the outputs.

When AutoGen and RAG are integrated, particularly with a vector store like ChromaDB, the synergy is striking. A vector store, such as ChromaDB, is a specialized database that effici